In [1]:
from elasticsearch import Elasticsearch, exceptions
import json, time


In [2]:
DOMAIN = "localhost"
PORT = 9200

In [3]:
host = str(DOMAIN) + ":" + str(PORT)

In [4]:
client = Elasticsearch(host)

In [5]:
try:
    # use the JSON library's dump() method for indentation
    info = json.dumps(client.info(), indent=4)

    # pass client object to info() method
    print ("Elasticsearch client info():", info)

except exceptions.ConnectionError as err:

    # print ConnectionError for Elasticsearch
    print ("\nElasticsearch info() ERROR:", err)
    print ("\nThe client host:", host, "is invalid or cluster is not running")

    # change the client's value to 'None' if ConnectionError
    client = None

Elasticsearch client info(): {
    "cluster_name": "elasticsearch",
    "name": "elk",
    "tagline": "You Know, for Search",
    "version": {
        "minimum_wire_compatibility_version": "6.8.0",
        "number": "7.4.0",
        "build_flavor": "default",
        "minimum_index_compatibility_version": "6.0.0-beta1",
        "build_snapshot": false,
        "build_date": "2019-09-27T08:36:48.569419Z",
        "lucene_version": "8.2.0",
        "build_hash": "22e1767283e61a198cb4db791ea66e3f11ab9910",
        "build_type": "tar"
    },
    "cluster_uuid": "U3E6ysB6SHWDQs0RFHccDA"
}


In [6]:
if client == None:
    exit()

In [7]:
print(client.indices.get_alias("*"))

{'.kibana_1': {'aliases': {'.kibana': {}}}, 'twitter-corona': {'aliases': {}}, '.kibana_task_manager_1': {'aliases': {'.kibana_task_manager': {}}}, 'twitter-corona-3.': {'aliases': {}}, 'twitter': {'aliases': {}}, 'twitter-corona-': {'aliases': {}}, '.apm-agent-configuration': {'aliases': {}}}


In [8]:
import json

In [22]:
#last scan on 28th April
# indices = ['twitter-corona-3.','twitter-corona','twitter-corona-']
indices = ['twitter-corona']
doc_count = 0
filecount = 1
filename = ""
PATH = "./dataset/Data/"
f = None
data = {}
# iterate over the list of Elasticsearch indices
for num, index in enumerate(indices):

    # declare a filter query dict object
    match_all = {
#         "_source": ["tweet_id","tweet_created_at","tweet_content","author.author_location"],
        "_source": ["tweet_id","tweet_created_at"],
        "size": 10000,
#         "sort": ['tweet_created_at'],
        "query": {
#             "match_phrase": {"author.author_location": "India"}
            "match_all": {}
        }
    }
    
#     match_all = {
#             "size": 1000,
#         "sort": ['tweet_created_at'],
#         "query": {
#             "match_all": {}
#         }
#     }
    
    
    # make a search() request to get all docs in the index
    resp = client.search(
        index = index,
        body = match_all,
        scroll = '5s' # length of time to keep search context
    )

    # keep track of pass scroll _id
    old_scroll_id = resp['_scroll_id']

    # use a 'while' iterator to loop over document 'hits'
    while len(resp['hits']['hits']):

        # make a request using the Scroll API
        resp = client.scroll(
            scroll_id = old_scroll_id,
            scroll = '5s' # length of time to keep search context
        )

        # check if there's a new scroll ID
        if old_scroll_id != resp['_scroll_id']:
            print ("NEW SCROLL ID:", resp['_scroll_id'])

        # keep track of pass scroll _id
        old_scroll_id = resp['_scroll_id']

        # print the response results
#         print ("\nresponse for index:", index)
#         print ("_scroll_id:", resp['_scroll_id'])
#         print ('response["hits"]["total"]["value"]:', resp["hits"]["total"]["value"])
        # iterate over the document hits for each 'scroll'
        for doc in resp['hits']['hits']:
            filename = str(resp['hits']['hits'][0]['_source']['tweet_created_at'][:10]) + ".txt"
            if filename in data.keys():
                data[filename].append(doc['_id']+"\n")
            else:
                data[filename] = []
                data[filename].append(doc['_id']+"\n")
            doc_count += 1
        if (doc_count % 100000) == 0:
            #for all the files
            count = 0
            for key in data.keys():
                f = open(PATH + key, "a")
                #write all the data of the file
                for ids in data[key]:
                    count = count + 1
                    f.write(ids)
                f.close()
                #refresh the buffer
                data[key] = []
            print(count)
            print ("DOC COUNT:", doc_count)  

100000
DOC COUNT: 100000
100000
DOC COUNT: 200000
100000
DOC COUNT: 300000
100000
DOC COUNT: 400000
100000
DOC COUNT: 500000
100000
DOC COUNT: 600000
100000
DOC COUNT: 700000
100000
DOC COUNT: 800000
100000
DOC COUNT: 900000
100000
DOC COUNT: 1000000
100000
DOC COUNT: 1100000
100000
DOC COUNT: 1200000
100000
DOC COUNT: 1300000
100000
DOC COUNT: 1400000
100000
DOC COUNT: 1500000
100000
DOC COUNT: 1600000
100000
DOC COUNT: 1700000
100000
DOC COUNT: 1800000
100000
DOC COUNT: 1900000
100000
DOC COUNT: 2000000
100000
DOC COUNT: 2100000
100000
DOC COUNT: 2200000
100000
DOC COUNT: 2300000
100000
DOC COUNT: 2400000
100000
DOC COUNT: 2500000
100000
DOC COUNT: 2600000
100000
DOC COUNT: 2700000
100000
DOC COUNT: 2800000
100000
DOC COUNT: 2900000
100000
DOC COUNT: 3000000
100000
DOC COUNT: 3100000
100000
DOC COUNT: 3200000
100000
DOC COUNT: 3300000
100000
DOC COUNT: 3400000
100000
DOC COUNT: 3500000
100000
DOC COUNT: 3600000
100000
DOC COUNT: 3700000
100000
DOC COUNT: 3800000
100000
DOC COUNT: 390

100000
DOC COUNT: 30900000
100000
DOC COUNT: 31000000
100000
DOC COUNT: 31100000
100000
DOC COUNT: 31200000
100000
DOC COUNT: 31300000
100000
DOC COUNT: 31400000
100000
DOC COUNT: 31500000
100000
DOC COUNT: 31600000
100000
DOC COUNT: 31700000
100000
DOC COUNT: 31800000
100000
DOC COUNT: 31900000
100000
DOC COUNT: 32000000
100000
DOC COUNT: 32100000
100000
DOC COUNT: 32200000
100000
DOC COUNT: 32300000
100000
DOC COUNT: 32400000
100000
DOC COUNT: 32500000
100000
DOC COUNT: 32600000
100000
DOC COUNT: 32700000
100000
DOC COUNT: 32800000
100000
DOC COUNT: 32900000
100000
DOC COUNT: 33000000
100000
DOC COUNT: 33100000
100000
DOC COUNT: 33200000
100000
DOC COUNT: 33300000
100000
DOC COUNT: 33400000
100000
DOC COUNT: 33500000
100000
DOC COUNT: 33600000
100000
DOC COUNT: 33700000
100000
DOC COUNT: 33800000
100000
DOC COUNT: 33900000
100000
DOC COUNT: 34000000
100000
DOC COUNT: 34100000
100000
DOC COUNT: 34200000
100000
DOC COUNT: 34300000
100000
DOC COUNT: 34400000
100000
DOC COUNT: 34500000
1

100000
DOC COUNT: 61300000
100000
DOC COUNT: 61400000
100000
DOC COUNT: 61500000
100000
DOC COUNT: 61600000
100000
DOC COUNT: 61700000
100000
DOC COUNT: 61800000
100000
DOC COUNT: 61900000
100000
DOC COUNT: 62000000
100000
DOC COUNT: 62100000
100000
DOC COUNT: 62200000
100000
DOC COUNT: 62300000
100000
DOC COUNT: 62400000
100000
DOC COUNT: 62500000
100000
DOC COUNT: 62600000
100000
DOC COUNT: 62700000
100000
DOC COUNT: 62800000
100000
DOC COUNT: 62900000
100000
DOC COUNT: 63000000
100000
DOC COUNT: 63100000
100000
DOC COUNT: 63200000
100000
DOC COUNT: 63300000
100000
DOC COUNT: 63400000
100000
DOC COUNT: 63500000
100000
DOC COUNT: 63600000
100000
DOC COUNT: 63700000
100000
DOC COUNT: 63800000
100000
DOC COUNT: 63900000
100000
DOC COUNT: 64000000
100000
DOC COUNT: 64100000
100000
DOC COUNT: 64200000
100000
DOC COUNT: 64300000
100000
DOC COUNT: 64400000
100000
DOC COUNT: 64500000
100000
DOC COUNT: 64600000
100000
DOC COUNT: 64700000
100000
DOC COUNT: 64800000
100000
DOC COUNT: 64900000
1